In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 895.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 54.6 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece
!pip install langchain-community langchain-core
!pip install --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.7 MB/s eta 0:00:00


In [3]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "Universal-NER/UniNER-7B-all"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    max_length=3000,
    do_sample=True,
    top_k=3,
    max_new_tokens=1000,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/766 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [13]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [14]:
from langchain import PromptTemplate,  LLMChain

template = """
              A virtual assistant answers questions from a user based on the provided text.
              USER: Text: {input_text}
              ASSISTANT: I’ve read this text.
              USER: What describes {entity_type} in the text?
              ASSISTANT:
           """

prompt = PromptTemplate(template=template, input_variables=["input_text","entity_type"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [17]:
text2 ="""Hetarth Chopra
447-902-0475 | hetarth2@illinois.edu | https://www.linkedin.com/in/hetarth-chopra/ | https://github.com/choprahetarth
Education
University of Illinois at Urbana Champaign Champaign, IL
Master’s of Science (MS) in Statistics; GPA - 4/4 Aug. 2023 – May 2025
• Coursework - Statistical Modelling (Data Analytics), Probability and Statistics, Statistical Learning (Machine Learning).
Thapar University Patiala, India
Bachelor’s of Engineering (B.E.) in Mechatronics; GPA - 8.62/10 Aug. 2016 – May 2020
• Coursework - Calculus, Linear Algebra, Numerical Analysis, C++, Computer Architecture, Robotics (Artificial Intelligence).
Experience
Graduate Research Assistant Aug. 2023 – Present
AIFARMS (and IBM), University of Illinois Skills - Large Language Models, PyTorch, Python Champaign, IL, USA
• Surpassed GPT-4 quality by 31%, after fine-tuning a code generation LLM on Ansible Data, utilizing QLoRA.
• Collaborated with AIFARMS and applied Topic - Clustering to create a 3.8GB Agricultural Document Corpus.
Machine Learning Engineer (NLP) Oct. 2022 – Jul. 2023
Cactus Communications Skills - Tensorflow, Spark, Numpy, Langchain, R-Studio, Postgresql Remote, India
• Mentored a team of 2 interns for notching up precision of language evaluation by 38% of RoBERTa based infrastructure.
• Partnered with Project Managers to develop an R-based Regression model, saving USD 1.3 million in payments.
• Presented to CTO - Data-Driven Bayesian Hyperparameter tuning for expanding accuracy of payments model by 89%.
Jr. Machine Learning Engineer (CV and NLP) Nov. 2021 – Oct. 2022
Dukaan Skills - Python, PyTorch, Flask, Pandas, Scikit-Learn, AWS, Statistical Modelling Bengaluru, India
• Enhanced image clarity boosting Peak Signal-to-Noise Ratio by 72% by leveraging DeblurGAN in the LAB color space.
• Captured 32% more frauds by estimating a product’s price using a vector database with MiniLM (Quantized Sentence LLM),
Clustering, semantic search, and statistical Z-Score outlier detection, .
• Conducted hypothesis testing to optimize user experience, resulting in a 27% uplift in user engagement.
• Diminished Data Latency for Fraud Detection API by 40%, by using Apache Kafka, RabbitMQ and Apache Spark.
Graduate Engineer Trainee (Data Science) Nov. 2020 – Sep. 2021
Alstom- Skills - Excel, Natural Language Processing, XGBoost, Spark, Data Visualization, Tableau Bengaluru, India
• Deployed a requirement classification tool using Neural Networks, diminishing categorization time by 3 hours.
• Employed Matplotlib, Seaborn for an internal Python Dashboard showcasing 12 KPI’s in real-time.
• Extracted and applied data cleaning (ETL) for 12 million rows RFP Dataset provisioning PySpark.
Skills
Languages and Frameworks: Python, C++, SQL, HTML, Java, Javascript, R, SAS, TensorFlow, PyTorch, OpenCV, Flask
Concepts: Computer Science, Data Science, Data Analysis, LLM’s, Machine Learning, NLP, Computer Vision, Quantitative
Methods, Data Structures and Algorithms, Containerization, Image Processing, Model Training, A/B Testing.
Tools and Libraries: Git, Javascript, AWS, Docker, Kubernetes, Selenium, ELK Stack, Spark, Hadoop, Docker, Tableau,
Jupyter Notebook, Pandas, NumPy, Matplotlib, HuggingFace, Numpy, Scipy, XGBoost, ScikitLearn, Linux
Soft Skills: Strong Communication Skills, Analytical and Problem Solving Mindset, Proficiency in Microsoft Excel, Microsoft
Word, Microsoft Powerpoint, Adobe Photoshop
Achievements and Extra Activities
Kaggle Expert - Notebooks May 2022 - Nov. 2022
Gold Medalist & Scholarship - Thapar University, Patiala Feb. 2019 - Feb. 2020"""


entity_type="Sections"
print(llm_chain.run({"input_text":text2,"entity_type":entity_type}))

Both `max_new_tokens` (=1000) and `max_length`(=3000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



              A virtual assistant answers questions from a user based on the provided text.
              USER: Text: Hetarth Chopra
447-902-0475 | hetarth2@illinois.edu | https://www.linkedin.com/in/hetarth-chopra/ | https://github.com/choprahetarth
Education
University of Illinois at Urbana Champaign Champaign, IL
Master’s of Science (MS) in Statistics; GPA - 4/4 Aug. 2023 – May 2025
• Coursework - Statistical Modelling (Data Analytics), Probability and Statistics, Statistical Learning (Machine Learning).
Thapar University Patiala, India
Bachelor’s of Engineering (B.E.) in Mechatronics; GPA - 8.62/10 Aug. 2016 – May 2020
• Coursework - Calculus, Linear Algebra, Numerical Analysis, C++, Computer Architecture, Robotics (Artificial Intelligence).
Experience
Graduate Research Assistant Aug. 2023 – Present
AIFARMS (and IBM), University of Illinois Skills - Large Language Models, PyTorch, Python Champaign, IL, USA
• Surpassed GPT-4 quality by 31%, after fine-tuning a code generation LLM o